# data validation 
- datatypes 
- data drift 
- same schema
- validate column


In [1]:
# constants 
""" 
Data Validation related constants 
"""
DATA_VALIDATION_DIR_NAME: str = 'data_validation'
DATA_VALIDATION_VALID_DIR: str = 'validated'
DATA_VALIDATION_INVALID_DIR: str = 'invalid'
DATA_VALIDATION_DRIFT_REPORT_DIR: str = 'drift_report'
DATA_VALIDATION_DRIFT_REPRT_FILE_NAME: str = 'report.yaml'

In [5]:
# config 
from NetworkSecurity.constants import training_pipeline
from NetworkSecurity.config.configuration import TrainingPipelineConfig
import os 

class DataValidationConfig: 
    def __init__(self,training_pipeline_config:TrainingPipelineConfig):
        self.data_validation_dir: str = os.path.join(training_pipeline_config.artifact_dir,training_pipeline.DATA_VALIDATION_DIR_NAME)
        self.valid_data_dir: str = os.path.join(self.data_validation_dir,training_pipeline.DATA_VALIDATION_VALID_DIR)
        self.invalid_data_dir: str = os.path.join(self.data_validation_dir, training_pipeline.DATA_VALIDATION_INVALID_DIR)
        self.valid_train_file_path: str =os.path.join(self.valid_data_dir, training_pipeline.TRAIN_FILE_NAME)
        self.valid_test_file_path: str = os.path.join(self.valid_data_dir, training_pipeline.TEST_FILE_NAME)
        self.invalid_train_file_path: str = os.path.join(self.invalid_data_dir, training_pipeline.TRAIN_FILE_NAME)
        self.invalid_test_file_path: str = os.path.join(self.invalid_data_dir, training_pipeline.TEST_FILE_NAME)
        self.drift_report_file_path: str = os.path.join(self.data_validation_dir, 
                                                        training_pipeline.DATA_VALIDATION_DRIFT_REPORT_DIR, 
                                                        training_pipeline.DATA_VALIDATION_DRIFT_REPRT_FILE_NAME
                                                        )

In [ ]:
# component 
from NetworkSecurity.entity.artifact_entity import DataIngestionArtifact

